In [1]:
import pandas as pd
import spacy
import os
import spacy
import polars as pl
import mlflow
import mlflow.spacy
from pathlib import Path
import json
from spacy.util import minibatch, compounding
from spacy.cli.train import train
from dotenv import load_dotenv
import sys
sys.path.insert(0, os.path.abspath('..'))
sys.path.insert(0, os.path.abspath('../..'))
from app.data_handling.preprocessing import process_ingredient, transform_ingredients_to_tokens

# Create Dataframe and save as parquet

In [2]:
load_dotenv()

True

In [3]:
mlflow.set_tracking_uri(os.getenv("MLFLOW_TRACKING_URL"))

In [4]:
model_uri = "models:/recipe_NER@prod"
ner = mlflow.spacy.load_model(model_uri=model_uri)

/home/naabzer/develop/recipe_tm/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [371]:
nlg_ds = pd.read_csv('../../data/recipenlg/RecipeNLG_dataset.csv', converters={'ingredients': pd.eval}, nrows=10)

In [397]:
nlg_ds = pl.read_csv('../../data/recipenlg/RecipeNLG_dataset.csv')#,  n_rows=1000)
nlg_ds = nlg_ds.with_columns(pl.col("ingredients").str.json_decode())
nlg_ds = nlg_ds.with_columns(pl.col("directions").str.json_decode())
nlg_ds = nlg_ds.with_columns(pl.arange(pl.len()).alias("index"))

In [401]:
nlg_ds

,title,ingredients,directions,link,source,NER,index
i64,str,list[str],list[str],str,str,str,i64
0,"""No-Bake Nut Cookies""","[""1 c. firmly packed brown sugar"", ""1/2 c. evaporated milk"", … ""3 1/2 c. bite size shredded rice biscuits""]","[""In a heavy 2-quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine."", ""Stir over medium heat until mixture bubbles all over top."", … ""Let stand until firm, about 30 minutes.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""brown sugar"", ""milk"", ""vanil…",0
1,"""Jewell Ball'S Chicken""","[""1 small jar chipped beef, cut up"", ""4 boned chicken breasts"", … ""1 carton sour cream""]","[""Place chipped beef on bottom of baking dish."", ""Place chicken on top of beef."", ""Mix soup and cream together; pour over chicken. Bake, uncovered, at 275° for 3 hours.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""beef"", ""chicken breasts"", ""c…",1
2,"""Creamy Corn""","[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg. cream cheese, cubed"", … ""1/4 tsp. pepper""]","[""In a slow cooker, combine all ingredients. Cover and cook on low for 4 hours or until heated through and cheese is melted. Stir well before serving. Yields 6 servings.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""frozen corn"", ""cream cheese""…",2
3,"""Chicken Funny""","[""1 large whole chicken"", ""2 (10 1/2 oz.) cans chicken gravy"", … ""4 oz. shredded cheese""]","[""Boil and debone chicken."", ""Put bite size pieces in average size square casserole dish."", … ""Sprinkle shredded cheese on top and bake at 350° for approximately 20 minutes or until golden and bubbly.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""chicken"", ""chicken gravy"", ""…",3
4,"""Reeses Cups(Candy) ""","[""1 c. peanut butter"", ""3/4 c. graham cracker crumbs"", … ""1 large pkg. chocolate chips""]","[""Combine first four ingredients and press in 13 x 9-inch ungreased pan."", ""Melt chocolate chips and spread over mixture. Refrigerate for about 20 minutes and cut into pieces before chocolate gets hard."", ""Keep in refrigerator.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""peanut butter"", ""graham crac…",4
…,…,…,…,…,…,…,…
2231137,"""Sunny's Fake Crepes""","[""1/2 cup chocolate hazelnut spread (recommended: Nutella)"", ""4 (8-inch) round whole-wheat tortillas, room temperature"", … ""1/2 cup finely chopped toasted hazelnuts""]","[""Spread hazelnut spread on 1 side of each tortilla."", ""In a large pan over medium heat add 2 tablespoons of butter and let it melt until bubbly."", … ""Arrange them on a serving platter and serve immediately.""]","""www.foodnetwork.com/recipes/su…","""Recipes1M""","""[""chocolate hazelnut spread"", …",2231137
2231138,"""Devil Eggs""","[""1 dozen eggs"", ""1 paprika"", … ""3 tbsp relish""]","[""Boil eggs on medium for 30mins."", ""Then cool eggs down in cold water for approximately 5 mins."", … ""finish touch sprinkle paprika on top.""]","""cookpad.com/us/recipes/355411-…","""Recipes1M""","""[""eggs"", ""paprika"", ""salt"", ""c…",2231138
2231139,"""Extremely Easy and Quick - Nam…","[""150 grams Daikon radish"", ""1 tbsp Sesame oil"", … ""1 tsp Soy sauce""]","[""Julienne the daikon and squeeze out the excess moisture."", ""Combine all of the ingredients."", ""Serve on a dish.""]","""cookpad.com/us/recipes/153324-…","""Recipes1M""","""[""radish"", ""Sesame oil"", ""Whit…",2231139


In [375]:
def transform_ds_to_BOW(ds: pl.dataframe.frame.DataFrame, create_vocab=False):  # Assume recipeNLG dataset, where each ingredient line is a list item
    tokens = []
    preps = []
    optionals = []
    for i, ingredients in enumerate(ds['ingredients']):
        datapoint_obj = transform_ingredients_to_tokens(ingredients, ner, create_vocab)
        tokens.append(datapoint_obj['foods'])
        preps.append(json.dumps(datapoint_obj['preps']))
        optionals.append(datapoint_obj['optionals'])
    new_ds = ds.with_columns(
        pl.Series(name='BOW_tokens', values=tokens),
        pl.Series(name='preps', values=preps),
        pl.Series(name='optionals', values=optionals)
    )
    return new_ds

In [376]:
nlg_ds.head()

,title,ingredients,directions,link,source,NER
i64,str,list[str],list[str],str,str,str
0,"""No-Bake Nut Cookies""","[""1 c. firmly packed brown sugar"", ""1/2 c. evaporated milk"", … ""3 1/2 c. bite size shredded rice biscuits""]","[""In a heavy 2-quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine."", ""Stir over medium heat until mixture bubbles all over top."", … ""Let stand until firm, about 30 minutes.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""brown sugar"", ""milk"", ""vanil…"
1,"""Jewell Ball'S Chicken""","[""1 small jar chipped beef, cut up"", ""4 boned chicken breasts"", … ""1 carton sour cream""]","[""Place chipped beef on bottom of baking dish."", ""Place chicken on top of beef."", ""Mix soup and cream together; pour over chicken. Bake, uncovered, at 275° for 3 hours.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""beef"", ""chicken breasts"", ""c…"
2,"""Creamy Corn""","[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg. cream cheese, cubed"", … ""1/4 tsp. pepper""]","[""In a slow cooker, combine all ingredients. Cover and cook on low for 4 hours or until heated through and cheese is melted. Stir well before serving. Yields 6 servings.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""frozen corn"", ""cream cheese""…"
3,"""Chicken Funny""","[""1 large whole chicken"", ""2 (10 1/2 oz.) cans chicken gravy"", … ""4 oz. shredded cheese""]","[""Boil and debone chicken."", ""Put bite size pieces in average size square casserole dish."", … ""Sprinkle shredded cheese on top and bake at 350° for approximately 20 minutes or until golden and bubbly.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""chicken"", ""chicken gravy"", ""…"
4,"""Reeses Cups(Candy) ""","[""1 c. peanut butter"", ""3/4 c. graham cracker crumbs"", … ""1 large pkg. chocolate chips""]","[""Combine first four ingredients and press in 13 x 9-inch ungreased pan."", ""Melt chocolate chips and spread over mixture. Refrigerate for about 20 minutes and cut into pieces before chocolate gets hard."", ""Keep in refrigerator.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""peanut butter"", ""graham crac…"


In [377]:
nlg_ds = transform_ds_to_BOW(nlg_ds, create_vocab=True)

In [378]:
nlg_ds.write_parquet('test.parquet')

# Test some queries using DuckDB and parquet

In [379]:
import duckdb

In [380]:
duckdb.sql("SELECT * FROM 'test.parquet' AS tbl WHERE list_contains(tbl.BOW_tokens, 'egg')").pl()

C0,title,ingredients,directions,link,source,NER,BOW_tokens,preps,optionals
i64,str,list[str],list[str],str,str,str,list[str],str,list[bool]
6,"""Rhubarb Coffee Cake""","[""1 1/2 c. sugar"", ""1/2 c. butter"", … ""1 tsp. vanilla""]","[""Cream sugar and butter."", ""Add egg and beat well."", … ""Pour into greased 9 x 13-inch pan and add Topping.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""sugar"", ""butter"", ""egg"", ""bu…","[""sugar"", ""butter"", … ""rhubarb""]","""{""rhubarb"": [""finely"", ""cut""]}""","[false, false, … false]"
7,"""Scalloped Corn""","[""1 can cream-style corn"", ""1 can whole kernel corn"", … ""pepper to taste""]","[""Mix together both cans of corn, crackers, egg, 2 teaspoons of melted butter and pepper and place in a buttered baking dish."", ""Dot with remaining 4 teaspoons of butter."", ""Bake at 350° for 1 hour.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""cream-style corn"", ""whole ke…","[""cream_-_style"", ""corn"", … ""pepper""]","""{""saltine_cracker"": [""crush""],…","[false, false, … false]"
20,"""Grandma Hanrath'S Banana Bread…","[""1 c. sugar"", ""1/2 c. shortening"", … ""2 over-ripe bananas (chopped)""]","[""Cream sugar and shortening."", ""Add eggs, salt and soda, then bananas and flour."", … ""Bake in loaf pan at 350° for 1 hour.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""sugar"", ""shortening"", ""eggs""…","[""sugar"", ""egg"", … ""banana""]","""{""banana"": [""chop""]}""","[false, false, … false]"
21,"""Chocolate Frango Mints""","[""1 pkg. devil's food cake mix"", ""1 pkg. chocolate fudge pudding mix (instant)"", … ""6 oz. chopped Frango mints""]","[""Mix ingredients together for 5 minutes."", ""Scrape bowl often. Last fold in chocolate chip mints."", ""Bake at 350° for 35 to 40 minutes or until done (cake mix directions).""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""cake mix"", ""chocolate fudge …","[""cake_mix"", ""sour_cream"", ""egg""]","""{}""","[false, false, false]"
24,"""Prize-Winning Meat Loaf""","[""1 1/2 lb. ground beef"", ""1 c. tomato juice"", … ""1 1/2 tsp. salt""]","[""Mix well."", ""Press firmly into an 8 1/2 x 4 1/2 x 2 1/2-inch loaf pan."", … ""Makes 8 servings.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""ground beef"", ""tomato juice""…","[""beef"", ""tomato_juice"", … ""salt""]","""{""beef"": [""ground""], ""egg"": [""…","[false, false, … false]"
…,…,…,…,…,…,…,…,…,…
987,"""French Toast And Sauce""","[""4 eggs, beaten"", ""1/2 c. milk"", … ""8 slices day-old bread""]","[""In a wide shallow bowl, beat 1st four ingredients. For each slice, melt 1 tablespoon of butter in skillet. Dip bread in egg mixture; turn and coat the other side. Cook slices in butter, browning on both sides. Serve with Sauce as follows.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""eggs"", ""milk"", ""frozen lemon…","[""egg"", ""milk"", … ""bread""]","""{""egg"": [""beat""]}""","[false, false, … false]"
988,"""Spaghetti Pie""","[""6 oz. spaghetti"", ""2 Tbsp. olive oil"", … ""1/2 c. shredded Mozzarella (2 oz.)""]","[""Cook spaghetti (about 3 cups)."", ""Stir olive oil into hot spaghetti."", … ""Form spaghetti mixture into a ""crust"" in a 10-inch pie plate sprayed with Pam. Spread cottage cheese over bottom of spaghetti crust.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""spaghetti"", ""olive oil"", ""Pa…","[""olive_oil"", ""parmesan_cheese"", … ""mozzarella""]","""{""parmesan_cheese"": [""grate""],…","[false, false, … false]"
991,"""Favorite Chocolate Cake""","[""1 3/4 c. flour"", ""2 eggs"", … ""1 c. buttermilk""]","[""Sift all dry ingredients in a large mixing bowl."", ""Stir."", … ""Tube pan takes 40 minutes.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""flour"", ""eggs"", ""baking powd…","[""egg"", ""bake_powder"", … ""buttermilk""]","""{}""","[false, false, … false]"


In [381]:
duckdb.sql("COPY(SELECT preps::JSON AS preps, * EXCLUDE (preps) FROM 'test.parquet') TO 'test.parquet' (FORMAT PARQUET, OVERWRITE TRUE)")

In [382]:
ingredients = ['butter']
preps = {'butter': ['cube'], 'cream_cheese': ['cube']}

In [383]:
list(preps.keys())

['butter', 'cream_cheese']

In [384]:
def construct_ingredient_query(pq_path: str, ingredients: list, preps: dict):
    prep_keys = list(preps.keys())
    base_sql = (f"SELECT *, json_extract(preps, {prep_keys}) AS prep_lists"
                f" FROM '{pq_path}'"
                " AS tbl"
                f" WHERE list_has_all(tbl.BOW_tokens, {ingredients})"
               )
    prep_filter = ""
    for i, (key, value) in enumerate(preps.items()):
        prep_filter += (f" AND (prep_lists->>{i}) IS NOT NULL"
                        f" AND list_has_all(from_json(prep_lists->>{i}, '[\"VARCHAR\"]'), {value})"
                       )
    sql = base_sql + prep_filter
    return sql

In [385]:
duckdb.sql(construct_ingredient_query('test.parquet', ingredients, preps)).pl()

preps,C0,title,ingredients,directions,link,source,NER,BOW_tokens,optionals,prep_lists
str,i64,str,list[str],list[str],str,str,str,list[str],list[bool],list[str]
"""{""cream_cheese"": [""cube""], ""bu…",2,"""Creamy Corn""","[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg. cream cheese, cubed"", … ""1/4 tsp. pepper""]","[""In a slow cooker, combine all ingredients. Cover and cook on low for 4 hours or until heated through and cheese is melted. Stir well before serving. Yields 6 servings.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""frozen corn"", ""cream cheese""…","[""corn"", ""cream_cheese"", … ""pepper""]","[false, false, … false]","[""[""cube""]"", ""[""cube""]""]"


In [386]:
ingredients = ['almond']
preps = {}
sql = construct_ingredient_query('test.parquet', ingredients, preps)
results = duckdb.sql(sql).pl()
print("###############################")
print("####### WITHOUT PREPS #########")
print("###############################")
for title, ingr in results['title', 'ingredients'].iter_rows():
    print()
    print(f"---- {title} ----")
    print("\n".join(ingr))
    
ingredients = ['almond']
preps = {'almond': ['chop']}
sql = construct_ingredient_query('test.parquet', ingredients, preps)
results = duckdb.sql(sql).pl()
print("##############################")
print("######## WITH PREPS ##########")
print("##############################")
for title, ingr in results['title', 'ingredients'].iter_rows():
    print()
    print(f"---- {title} ----")
    print("\n".join(ingr))

###############################
####### WITHOUT PREPS #########
###############################

---- Honey Almond Squares ----
1 Duncan Hines white pudding cake mix
1 (8 oz.) pkg. cream cheese (at room temperature)
1/4 c. alfalfa honey
3 to 4 oz. sliced almonds

---- Caramel Pie ----
16 oz. Cool Whip
2 c. coconut
3/4 margarine, melted
1 c. slivered almonds, chopped fine
1 can Eagle Brand milk
3 graham cracker crusts
8 oz. cream cheese
1 jar caramel topping

---- Almond Roco ----
1 lb. butter (not margarine)
2 c. granulated sugar
1/2 c. water
1/2 c. chopped almonds
3 chocolate candy bars
1/4 c. chopped almonds or as desired (for topping)

---- Crabmeat Au Gratin ----
1 can cream of shrimp soup
1/2 can milk (use soup can)
1 lb. faux crabmeat
4 c. celery, chopped
1/2 c. green pepper, chopped
2 pimentos, chopped
2 Tbsp. onions, chopped
1/3 c. slivered almonds
4 hard-cooked eggs, chopped
1 c. shredded sharp Cheddar cheese
buttered bread crumbs

---- Fresh Broccoli Salad ----
2 bunches fres

In [388]:
nlg_ds.schema['index']

Int64

In [389]:
nlg_ds

,title,ingredients,directions,link,source,NER,BOW_tokens,preps,optionals,index
i64,str,list[str],list[str],str,str,str,list[str],str,list[bool],i64
0,"""No-Bake Nut Cookies""","[""1 c. firmly packed brown sugar"", ""1/2 c. evaporated milk"", … ""3 1/2 c. bite size shredded rice biscuits""]","[""In a heavy 2-quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine."", ""Stir over medium heat until mixture bubbles all over top."", … ""Let stand until firm, about 30 minutes.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""brown sugar"", ""milk"", ""vanil…","[""brown_sugar"", ""milk"", … ""butter""]","""{}""","[false, false, … false]",0
1,"""Jewell Ball'S Chicken""","[""1 small jar chipped beef, cut up"", ""4 boned chicken breasts"", … ""1 carton sour cream""]","[""Place chipped beef on bottom of baking dish."", ""Place chicken on top of beef."", ""Mix soup and cream together; pour over chicken. Bake, uncovered, at 275° for 3 hours.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""beef"", ""chicken breasts"", ""c…","[""beef"", ""chicken_breast"", … ""sour_cream""]","""{""beef"": [""chip"", ""cut"", ""up""]…","[false, false, … false]",1
2,"""Creamy Corn""","[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg. cream cheese, cubed"", … ""1/4 tsp. pepper""]","[""In a slow cooker, combine all ingredients. Cover and cook on low for 4 hours or until heated through and cheese is melted. Stir well before serving. Yields 6 servings.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""frozen corn"", ""cream cheese""…","[""corn"", ""cream_cheese"", … ""pepper""]","""{""cream_cheese"": [""cube""], ""bu…","[false, false, … false]",2
3,"""Chicken Funny""","[""1 large whole chicken"", ""2 (10 1/2 oz.) cans chicken gravy"", … ""4 oz. shredded cheese""]","[""Boil and debone chicken."", ""Put bite size pieces in average size square casserole dish."", … ""Sprinkle shredded cheese on top and bake at 350° for approximately 20 minutes or until golden and bubbly.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""chicken"", ""chicken gravy"", ""…","[""chicken"", ""chicken_gravy"", ""cream_of_mushroom_soup""]","""{}""","[false, false, false]",3
4,"""Reeses Cups(Candy) ""","[""1 c. peanut butter"", ""3/4 c. graham cracker crumbs"", … ""1 large pkg. chocolate chips""]","[""Combine first four ingredients and press in 13 x 9-inch ungreased pan."", ""Melt chocolate chips and spread over mixture. Refrigerate for about 20 minutes and cut into pieces before chocolate gets hard."", ""Keep in refrigerator.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""peanut butter"", ""graham crac…","[""peanut_butter"", ""graham_cracker_crumb"", … ""chocolate_chip""]","""{}""","[false, false, … false]",4
…,…,…,…,…,…,…,…,…,…,…
995,"""Heath Bar Pie""","[""3 Heath bars, chopped fine"", ""1 medium container whipping topping"", … ""chocolate curls or chopped nuts""]","[""Mix chopped Heath bars with whipped topping and pour into crust."", ""Top with chocolate curls or chopped nuts."", ""Refrigerate.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""graham cracker pie crust"", ""…","[""bar"", ""whip_topping"", … ""chocolate_curl""]","""{""bar"": [""chop""], ""chocolate_c…","[false, false, … false]",995
996,"""Victorian Baked French Toast""","[""1 c. brown sugar"", ""1/3 c. butter"", … ""1 tsp. cinnamon""]","[""Cook brown sugar, butter and corn syrup in small saucepan until butter melts."", ""Pour into 9 x 13-inch pan."", … ""Bake at 350° for 30 minutes or until center is done and lightly browned.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""brown sugar"", ""butter"", ""lig…","[""butter"", ""corn_syrup"", … ""cinnamon""]","""{}""","[false, false, … false]",996
997,"""Quick Swedish Meatballs""","[""1 lb. ground beef"", ""1 c. soft bread crumbs"", … ""1 pkg. stroganoff sauce mix""]","[""Combine meat, bread crumbs, cheese, soup mix, seasonings and 1/4 cup milk; mix thoroughly (with hands

In [415]:
import random
random.seed(123)
eval_data_idx = random.sample(range(0,nlg_ds.shape[0]), k=10000)

In [416]:
datanlg_ds.filter(pl.col("index").is_in(eval_data_idx))

,title,ingredients,directions,link,source,NER,index
i64,str,list[str],list[str],str,str,str,i64
493,"""Sour Cream Meat Loaf""","[""2 lb. ground lean beef"", ""1 1/2 c. grated raw carrot"", … ""2 c. sour cream""]","[""Combine all ingredients in large mixing bowl; blend well. Pack in a 9 x 5 x 3-inch loaf pan."", ""Bake at 350° for 1 1/2 to 1 3/4 hours."", ""Serves 6.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""ground lean beef"", ""carrot"",…",493
1244,"""Chocolate Pistachio Cake""","[""1 box Duncan Hines yellow cake mix"", ""1 box instant pistachio pudding"", … ""1/2 can Hershey's chocolate syrup""]","[""Blend to moisten, then beat for 2 minutes."", ""(Optional: a few drops of green food dye for a richer green color.) Pour about 3/4 batter into a greased and floured Bundt pan."", … ""When cool, dust with confectioners sugar.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""orange juice"", ""water"", ""oil…",1244
1292,"""Corn Pudding""","[""1 can golden crushed corn"", ""2 eggs, slightly beaten"", … ""sprinkle of paprika""]","[""Melt butter in casserole."", ""Beat eggs."", … ""Bake at 325° for an hour (it should jiggle and a knife should come out kinda clean).""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""golden crushed corn"", ""eggs""…",1292
1461,"""Apricot Cake""","[""2 c. sugar"", ""4 eggs"", … ""1/2 c. chopped nuts (English or pecans)""]","[""Mix together sugar, eggs, oil and baby food."", ""Add flour, baking soda, salt, cinnamon and nuts; bake at 350° for 1 hour and 20 minutes."", ""Make sure cake is completely cooled before frosting it.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""sugar"", ""eggs"", ""Wesson oil""…",1461
1487,"""Salmon Bake""","[""1 1/2 lb. salmon steaks"", ""dash of nutmeg"", ""1 green apple (a bit tart)""]","[""Preheat oven to 400° to 450°."", ""Wash and pat dry salmon steaks. Place on sheet of foil."", … ""Bake 15 to 25 minutes, covered, then unwrap and continue baking 5 to 10 minutes longer at lowered temperature of 350°, until salmon pulls into flakes with fork.""]","""www.cookbooks.com/Recipe-Detai…","""Gathered""","""[""salmon steaks"", ""nutmeg"", ""g…",1487
…,…,…,…,…,…,…,…
2229921,"""Parmesan Casserole Bread""","[""1 cup flour"", ""1 package dry yeast"", … ""1 -1 12 cup more flour""]","[""In a bowl combine the first cup of flour, yeast, sugar, and salt."", ""In a saucepan heat to about 115-120 degrees the milk, water, butter, and parsley and add this to the above."", … ""Bake in a 375 degree oven for 25-30 minutes.""]","""www.food.com/recipe/parmesan-c…","""Recipes1M""","""[""flour"", ""yeast"", ""sugar"", ""s…",2229921
2230086,"""Dairy-free Coconut Sherbet""","[""66 ml Raisins"", ""2 tbsp Rum"", … ""1 tsp Apple cider vinegar""]","[""Combine the rum and raisins into a small pot."", ""After bringing to a boil over medium heat, remove from heat and let the raisins absorb the rum as they cool."", … ""Put your finished product in a container with a lid and chill in the freezer.""]","""cookpad.com/us/recipes/151781-…","""Recipes1M""","""[""Raisins"", ""Coconut milk"", ""s…",2230086
2230490,"""Dolores Park Swizzle""","[""2 ounces white rhum agricole"", ""1/4 ounce maraschino liqueur"", … ""1 or 2 mint sprigs, for garnish""]","[""In a chilled collins glass, combine all of the ingredients except the ice and garnishes."", ""Add crushed ice."", … ""Top with the bitters and mint.""]","""www.foodandwine.com/recipes/do…","""Recipes1M""","""[""white rhum"", ""maraschino liq…",2230490


In [396]:
with open("test.json", "w") as f:
    json.dump(nlg_ds.slice(0,10).select('index', 'title', 'ingredients').write_json(), f)